In [ ]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sps
import scipy.optimize as optimize

# Searching for the Higgs boson

In [ ]:
Higgs_Mass = 125.1  # In Units of MeV / c**2
Higgs_Width = 4.2   # In Units of MeV / c**2

mass_grid = np.linspace(90., 160., 71)

In [ ]:
def passed_cuts(cut_width, masses, model_sig, model_bkg):
    mask = np.abs(masses - 125) < cut_width
    n_sig = np.sum(model_sig[mask])
    n_bkg = np.sum(model_bkg[mask])
    return n_sig, n_bkg

In [ ]:
def plotModels(masses, counts, ref_mass, init_pars):
    
    model_vals = model_func(masses, counts, ref_mass, *init_pars)
    background_vals = poly1(masses, counts, ref_mass, init_pars[3], init_pars[4])

    _ = plt.plot(masses, counts, label="data")
    _ = plt.plot(masses, background_vals, label="background model")
    _ = plt.plot(masses, model_vals, label="model")
    _ = plt.xlabel(r'$m [MeV / c^2]$')
    _ = plt.ylabel(r'Events [per GeV / $c^2$ / month]')
    _ = plt.legend()

In [ ]:
def plot_nexp_passed_cuts(masses, model_sig, model_bkg):
    sig_cts = np.zeros(26)
    bkg_cts = np.zeros(26)
    widths = np.linspace(0, 25, 26)
    for i, width in enumerate(widths):
        sig_cts[i], bkg_cts[i] = passed_cuts(width, masses, model_sig, model_bkg)
    _ = plt.plot(widths, sig_cts, label="Signal")
    _ = plt.plot(widths, bkg_cts, label="Background")
    _ = plt.yscale('log')
    _ = plt.xlabel(r"Cut Width [GeV / $c^2$]")
    _ = plt.ylabel(r"Events [per GeV / $c^2$ / month]")

In [ ]:
def find_sig2noise(mass_grid, model_sig, model_bkg, plot=True):
    sig_cts = np.zeros(26)
    bkg_cts = np.zeros(26)
    widths = np.linspace(0, 25, 26)
    for i, width in enumerate(widths):
        if i == 0:
            continue
        sig_cts[i], bkg_cts[i] = passed_cuts(width, mass_grid, model_sig, model_bkg)
    sig2noise = np.zeros(26)
    sig2noise[1:] = sig_cts[1:]/np.sqrt(bkg_cts[1:])
    if plot:
        _ = plt.plot(widths, sig_cts/np.sqrt(bkg_cts))
    return sig2noise

In [ ]:
def extract_peak_from_data(cut_width, masses, nevts):
    mask = np.abs(masses - 125) < cut_width
    return np.sum(nevts[mask])

In [ ]:
def estimate_bkg_from_data(masses, nevts, cut_width):
    mask_bkg_lo = np.abs(masses-105) < cut_width
    mask_bkg_hi = np.abs(masses-145) < cut_width
    mask_bkg = np.bitwise_or(mask_bkg_lo, mask_bkg_hi)
    bkg_estimate = 0.5 * np.sum(nevts[mask_bkg])
    return (bkg_estimate, np.sqrt(bkg_estimate))

In [ ]:
ref_mass = 130.
nsig_per_month = 20.
nbkg_per_mev_per_month = 40.
bkg_slope_per_mev_per_month = -0.2
model_bkg = poly1(mass_grid, ref_mass, nbkg_per_mev_per_month, bkg_slope_per_mev_per_month)
model_sig = Gauss(mass_grid, nsig_per_month, Higgs_Mass, Higgs_Width)

In [ ]:
_ = plt.plot(mass_grid, model_sig, label="Signal")
_ = plt.plot(mass_grid, model_bkg, label="Background")
_ = plt.xlabel(r"Mass [GeV/$c^2$]")
_ = plt.ylabel(r"Counts [per GeV/$c^2$]")

In [ ]:
plot_nexp_passed_cuts(mass_grid, model_sig, model_bkg)

In [ ]:
_ = find_sig2noise(mass_grid, model_sig, model_bkg)

In [ ]:
max_s2n = np.zeros(24)
best_cut = np.zeros(24)
n_months_array = np.arange(24)
for n_months in n_months_array:
    if n_months == 0:
        continue
    s2n = find_sig2noise(mass_grid, n_months*model_sig, n_months*model_bkg, plot=False)
    max_s2n[n_months] = np.max(s2n)
    best_cut[n_months] = np.argmax(s2n)
_ = plt.scatter(n_months_array, max_s2n)

In [ ]:
_ = plt.plot(mass_grid, 24*(model_sig+model_bkg))

In [ ]:
#tot_model = np.random.poisson(24*(model_sig+model_bkg))
#try:
#    os.unlink('Higgs.txt')
#except FileNotFoundError:
#    pass
#fout = open('Higgs.txt', 'w')
#fout.write("# Mass     Nevts\n")
#for mass, nevt in zip(mass_grid, tot_model):
#    fout.write("%0.1f %i\n" % (mass, nevt))
#fout.close()

In [ ]:
data = np.loadtxt('Higgs.txt')
masses = data[:,0]
nevts = data[:,1]
errors = np.sqrt(nevts)
_ = plt.errorbar(masses, nevts, yerr=errors, fmt='.')

In [ ]:
n_peak = extract_peak_from_data(6, masses, nevts)
print(n_peak)
n_bkg, sigma_bkg = estimate_bkg_from_data(masses, nevts, 6)
print(n_bkg, sigma_bkg)
n_sigma = (n_peak - n_bkg)/sigma_bkg
print(n_sigma)

In [ ]:
from functools import partial

def Gauss(x, nsig, mu, sigma):
    return nsig*sps.norm(loc=mu, scale=sigma).pdf(x)

def poly1(x, ref_mass, offset, slope):
    return offset + (x-ref_mass)*slope

def model_func(x, ref_mass, nsig, offset, slope):
    return Gauss(x, nsig, Higgs_Mass, Higgs_Width) + poly1(x, ref_mass, offset, slope)

def generic_chi2(params, data_vals, model, x, ref_mass):
    model_vals = model(x, ref_mass, *params)
    return np.sum(((data_vals - model_vals)**2)/data_vals)

def cost_func(data_vals, model, x, ref_mass):
    return partial(generic_chi2, data_vals=data_vals, model=model, x=x, ref_mass=ref_mass)

In [ ]:
def fitAndPlotResult(masses, nevts, ref_mass, init_pars):
    our_cost_func = cost_func(nevts, model_func, masses, ref_mass=ref_mass)
    result = optimize.minimize(our_cost_func, x0=np.array(init_pars))
    fit_pars = result['x']
    cov = result['hess_inv']
    model_fit = model_func(masses, ref_mass, *fit_pars)
    background_fit = poly1(masses, ref_mass, fit_pars[1], fit_pars[2])
    print("Best Fit ---------")
    print("N Signal: %.1f [Events]" % fit_pars[0])
    print("Higgs Peak: %.4f [GeV]" % Higgs_Mass)
    print("Higgs Width: %.4f [GeV]" % Higgs_Width)
    print("Background at 125 GeV: %.2f [Events / GeV]" % fit_pars[1])
    print("Background slope: %.2f [Events / GeV / GeV]" % fit_pars[2])
    _ = plt.errorbar(masses, nevts, yerr=np.sqrt(nevts), fmt='.', label="data")
    _ = plt.plot(masses, background_fit, label="background model")
    _ = plt.plot(masses, model_fit, label="full model")
    _ = plt.xlabel(r"mass [GeV]")
    _ = plt.ylabel("Events [per GeV]")
    _ = plt.legend()
    print(cov)
    return (fit_pars[0], np.sqrt(cov[0,0]))

In [ ]:
init_pars = [200, 1000, -5.]

In [ ]:
fitAndPlotResult(masses, nevts, ref_mass, init_pars)